In [1]:
import os
import json
import pandas as pd
from glob import glob

def extract_notes_data_from_response(path, output_file):
    """
    读取指定路径下含"response"的文件，提取笔记数据并输出为Excel
    
    Args:
        path (str): 目标文件所在路径（例如："C:/data/response_files"）
        output_file (str): 输出Excel文件名（例如："notes_data.xlsx"）
    """
    # 初始化存储所有笔记数据的列表
    all_notes = []
    
    # 1. 匹配路径下所有文件名含"response"的文件（支持所有文件类型）
    response_files = glob(os.path.join(path, "*response*"))
    if not response_files:
        print(f"⚠️ 在路径 {path} 下未找到含'response'的文件")
        return
    
    print(f"✅ 找到 {len(response_files)} 个含'response'的文件，开始解析...")
    
    # 2. 遍历每个文件并提取数据
    for file_idx, file_path in enumerate(response_files, 1):
        file_name = os.path.basename(file_path)
        print(f"🔍 正在处理第 {file_idx}/{len(response_files)} 个文件：{file_name}")
        
        try:
            # 2.1 读取文件内容（处理编码异常）
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                content = f.read().strip()
            
            # 2.2 提取JSON部分（HTTP响应头后的数据，从第一个"{"到最后一个"}"）
            json_start = content.find("{")
            json_end = content.rfind("}")
            if json_start == -1 or json_end == -1:
                print(f"❌ {file_name} 中未找到有效JSON数据，跳过该文件")
                continue
            json_str = content[json_start:json_end + 1]
            
            # 2.3 解析JSON
            json_data = json.loads(json_str)
            
            # 2.4 检查JSON结构是否包含data->items
            if not (json_data.get("success") and "data" in json_data and "items" in json_data["data"]):
                print(f"❌ {file_name} 的JSON结构不完整（缺少data/items），跳过该文件")
                continue
            notes_items = json_data["data"]["items"]
            if not notes_items:
                print(f"ℹ️ {file_name} 中无笔记数据（items为空），跳过该文件")
                continue
        
        except Exception as e:
            print(f"❌ 处理 {file_name} 时出错：{str(e)}，跳过该文件")
            continue
        
        # 3. 提取每个笔记的目标字段
        for note in notes_items:
            # 3.1 基础笔记字段（处理缺失值）
            base_info = {
                "笔记ID": note.get("id", ""),
                "笔记类型(type)": note.get("type", ""),
                "笔记标题(title)": note.get("title", ""),
                "笔记描述(desc)": note.get("desc", ""),
                "发布时间戳(time)": note.get("time", 0),  # 后续转日期
                "笔记模型类型(model_type)": note.get("model_type", "")
            }
            
            # 3.2 作者信息（从user对象提取，处理user不存在的情况）
            user_info = note.get("user", {})
            author_info = {
                "作者ID(userid)": user_info.get("userid", ""),
                "作者用户名(name)": user_info.get("name", ""),
                "作者昵称(nickname)": user_info.get("nickname", ""),
                "作者头像URL(image)": user_info.get("image", ""),
                "是否关注作者(followed)": user_info.get("followed", False),
                "是否官方认证(red_official_verified)": user_info.get("red_official_verified", False)
            }
            
            # 3.3 互动数据模块（所有目标字段）
            interaction_info = {
                "点赞数(liked_count)": note.get("liked_count", 0),
                "收藏数(collected_count)": note.get("collected_count", 0),
                "分享数(shared_count)": note.get("shared_count", 0),
                "评论数(comments_count)": note.get("comments_count", 0),
                "浏览数(view_count)": note.get("view_count", 0),
                "当前用户是否点赞(liked)": note.get("liked", False),
                "当前用户是否收藏(collected)": note.get("collected", False)
            }
            
            # 3.4 分享链接与IP属地
            other_info = {
                "笔记分享链接(share_info.link)": note.get("share_info", {}).get("link", ""),
                "作者IP属地(ip_location)": note.get("ip_location", "")
            }
            
            # 3.5 合并所有字段为一条数据
            note_data = {**base_info, **author_info, **interaction_info, **other_info}
            all_notes.append(note_data)
    
    # 4. 处理数据并输出Excel
    if not all_notes:
        print("⚠️ 未提取到任何笔记数据，无需生成Excel")
        return
    
    # 4.1 创建DataFrame并转换时间戳为日期格式
    df = pd.DataFrame(all_notes)
    # 时间戳转日期（处理0值：转为空字符串，避免1970-01-01）
    df["发布时间(time)"] = pd.to_datetime(
        df["发布时间戳(time)"], 
        unit="s", 
        errors="coerce"  # 无效时间戳转为NaT
    ).dt.strftime("%Y-%m-%d %H:%M:%S")  # 格式化日期字符串
    # 删除原始时间戳列（可选，保留则注释此行）
    df = df.drop("发布时间戳(time)", axis=1)
    
    # 4.2 输出Excel（支持.xlsx格式，需安装openpyxl）
    try:
        df.to_excel(output_file, index=False, engine="openpyxl")
        print(f"🎉 数据提取完成！共 {len(df)} 条笔记数据，已保存至：{os.path.abspath(output_file)}")
    except ImportError:
        print("❌ 缺少Excel引擎依赖，请先执行：pip install openpyxl")
    except Exception as e:
        print(f"❌ 保存Excel时出错：{str(e)}")

# -------------------------- 配置参数（用户可修改） --------------------------
TARGET_PATH = r"C:\cao\code\1029\服美役\帖子\未命名文件夹2"  # 替换为你的文件路径
OUTPUT_EXCEL = "服美役.xlsx"  # 输出Excel文件名
# -----------------------------------------------------------------------------

# 执行函数
if __name__ == "__main__":
    extract_notes_data_from_response(TARGET_PATH, OUTPUT_EXCEL)

✅ 找到 10 个含'response'的文件，开始解析...
🔍 正在处理第 1/10 个文件：[56394] response_so.xiaohongshu.com_message.txt
🔍 正在处理第 2/10 个文件：[56405] response_so.xiaohongshu.com_message.txt
🔍 正在处理第 3/10 个文件：[56414] response_so.xiaohongshu.com_message.txt
🔍 正在处理第 4/10 个文件：[56433] response_so.xiaohongshu.com_message.txt
🔍 正在处理第 5/10 个文件：[56462] response_so.xiaohongshu.com_message.txt
🔍 正在处理第 6/10 个文件：[56491] response_so.xiaohongshu.com_message.txt
🔍 正在处理第 7/10 个文件：[56522] response_so.xiaohongshu.com_message.txt
🔍 正在处理第 8/10 个文件：[56557] response_so.xiaohongshu.com_message.txt
🔍 正在处理第 9/10 个文件：[56594] response_so.xiaohongshu.com_message.txt
🔍 正在处理第 10/10 个文件：[56628] response_so.xiaohongshu.com_message.txt
🎉 数据提取完成！共 200 条笔记数据，已保存至：c:\cao\code\xiaohongshu\服美役.xlsx


In [3]:
import os
import json
import pandas as pd

def timestamp_to_datetime(ts):
    """
    将时间戳转换为标准日期格式（兼容10位秒级/13位毫秒级时间戳）
    :param ts: 时间戳（int/str）
    :return: 格式化日期字符串（YYYY-MM-DD HH:MM:SS）或空字符串
    """
    if not ts:
        return ""
    ts_str = str(ts).strip()
    try:
        ts_int = int(ts_str)
        # 区分秒级（10位）和毫秒级（13位）时间戳
        if len(ts_str) == 10:
            return pd.to_datetime(ts_int, unit="s").strftime("%Y-%m-%d %H:%M:%S")
        elif len(ts_str) == 13:
            return pd.to_datetime(ts_int, unit="ms").strftime("%Y-%m-%d %H:%M:%S")
        else:
            return f"无效时间戳格式: {ts_str}"
    except Exception as e:
        return f"时间戳转换失败: {str(e)}"

def extract_response_notes(target_dir, output_xlsx):
    """
    提取指定目录下含"response"的文件中的笔记数据
    :param target_dir: 目标文件夹路径
    :param output_xlsx: 输出Excel文件名
    """
    # 存储所有笔记数据的列表
    all_notes_data = []
    
    # 1. 遍历目标目录，筛选含"response"的文件（不区分大小写）
    for filename in os.listdir(target_dir):
        if "response" in filename.lower():
            file_path = os.path.join(target_dir, filename)
            print(f"正在处理文件: {filename}")
            
            try:
                # 2. 读取文件内容（处理HTTP响应头+JSON混合格式）
                with open(file_path, "r", encoding="utf-8") as f:
                    file_content = f.read()
                
                # 提取JSON部分（从第一个"{"开始，忽略前面的HTTP响应头）
                json_start_idx = file_content.find("{")
                if json_start_idx == -1:
                    print(f"警告: {filename} 中未找到JSON数据，跳过该文件\n")
                    continue
                json_content = file_content[json_start_idx:]
                
                # 3. 解析JSON数据
                json_data = json.loads(json_content)
                # 检查数据结构是否合法（需包含data->items）
                if not json_data.get("success") or "data" not in json_data or "items" not in json_data["data"]:
                    print(f"警告: {filename} 中无有效items数据，跳过该文件\n")
                    continue
                notes_items = json_data["data"]["items"]
                
                # 4. 遍历每个笔记item，提取目标字段
                for item in notes_items:
                    # 4.1 笔记基础信息
                    note_base = {
                        "笔记ID": item.get("id", ""),
                        "笔记类型(type)": item.get("type", ""),
                        "笔记标题(title)": item.get("title", ""),
                        "笔记描述(desc)": item.get("desc", ""),
                        "笔记创建时间(note_time.create)": timestamp_to_datetime(item.get("note_time", {}).get("create_time")),
                        "笔记更新时间(note_time.update)": timestamp_to_datetime(item.get("note_time", {}).get("update_time"))
                    }
                    
                    # 4.2 作者信息（user对象）- 文档中无"name"字段，用nickname替代
                    author_info = {
                        "作者user_id": item.get("user", {}).get("user_id", ""),
                        "作者nickname": item.get("user", {}).get("nickname", ""),
                        "作者头像URL(avatar_url)": item.get("user", {}).get("avatar_url", "")
                    }
                    
                    # 4.3 互动数据（interaction_info所有字段，加前缀避免冲突）
                    interaction_data = item.get("interaction_info", {})
                    interaction_info = {f"互动_{k}": v for k, v in interaction_data.items()}
                    
                    # 4.4 分享链接（share_info.link）
                    share_link = {
                        "分享链接(share_info.link)": item.get("share_info", {}).get("link", "")
                    }
                    
                    # 4.5 合并所有字段为一条数据
                    single_note = {**note_base, **author_info, **interaction_info, **share_link}
                    all_notes_data.append(single_note)
                
                print(f"成功提取 {filename} 中的 {len(notes_items)} 条笔记数据\n")
            
            # 捕获文件处理中的异常（避免脚本崩溃）
            except UnicodeDecodeError:
                print(f"错误: {filename} 编码格式异常（非UTF-8），跳过该文件\n")
            except json.JSONDecodeError:
                print(f"错误: {filename} JSON解析失败，跳过该文件\n")
            except Exception as e:
                print(f"错误: 处理 {filename} 时发生未知错误 - {str(e)}，跳过该文件\n")
    
    # 5. 转换为DataFrame并保存为Excel
    if all_notes_data:
        df = pd.DataFrame(all_notes_data)
        # 使用openpyxl引擎保存（支持xlsx格式）
        df.to_excel(output_xlsx, index=False, engine="openpyxl")
        print(f"数据提取完成！共获取 {len(df)} 条笔记数据，已保存到 {output_xlsx}")
    else:
        print("未提取到任何有效笔记数据，请检查目标文件夹和文件格式")

# -------------------------- 配置参数（用户可修改）--------------------------
TARGET_DIRECTORY = r"C:\cao\code\1029\服美役\帖子\未命名文件夹2"  # 目标文件夹路径（含response文件的目录）
OUTPUT_EXCEL = "服美役.xlsx"  # 输出Excel文件名
# --------------------------------------------------------------------------

if __name__ == "__main__":
    # 检查目标文件夹是否存在
    if not os.path.exists(TARGET_DIRECTORY):
        print(f"错误: 目标文件夹 {TARGET_DIRECTORY} 不存在，请检查路径")
    else:
        # 安装依赖提示（首次运行若报错需执行）
        try:
            import openpyxl
        except ImportError:
            print("提示: 缺少openpyxl依赖，需先执行命令安装：pip install openpyxl pandas")
        else:
            extract_response_notes(TARGET_DIRECTORY, OUTPUT_EXCEL)

正在处理文件: [56394] response_so.xiaohongshu.com_message.txt
成功提取 [56394] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56405] response_so.xiaohongshu.com_message.txt
成功提取 [56405] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56414] response_so.xiaohongshu.com_message.txt
成功提取 [56414] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56433] response_so.xiaohongshu.com_message.txt
成功提取 [56433] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56462] response_so.xiaohongshu.com_message.txt
成功提取 [56462] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56491] response_so.xiaohongshu.com_message.txt
成功提取 [56491] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56522] response_so.xiaohongshu.com_message.txt
成功提取 [56522] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56557] response_so.xiaohongshu.com_message.txt
成功提取 [56557] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56594] response

In [13]:
import os
import json
from datetime import datetime
import pandas as pd

def extract_response_data(root_path, output_file):
    """
    提取指定路径下含"response"的文件数据，导出为Excel
    
    参数:
        root_path: str - 目标文件所在根路径（如 r"C:\your\file\path"）
        output_file: str - 输出Excel文件路径（如 r"C:\result.xlsx"）
    """
    # 初始化存储所有数据的列表（每行对应一条评论）
    all_data = []
    
    # 1. 遍历根路径下的所有文件
    for filename in os.listdir(root_path):
        # 筛选文件名包含"response"的文件（不区分大小写）
        if "response" in filename.lower():
            file_path = os.path.join(root_path, filename)
            print(f"正在处理文件: {file_path}")
            
            try:
                # 2. 读取文件内容（处理HTTP响应头+JSON格式）
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                # 提取JSON部分（跳过HTTP响应头，找到第一个"{"开始的内容）
                json_start_idx = content.find('{')
                if json_start_idx == -1:
                    print(f"警告: {filename} 中未找到JSON数据，跳过该文件\n")
                    continue
                json_str = content[json_start_idx:]
                
                # 3. 解析JSON数据
                json_data = json.loads(json_str)
                # 获取核心数据字段（处理字段缺失情况，用默认值兜底）
                data = json_data.get('data', {})
                base_user_id = data.get('user_id', '')  # data层的user_id
                base_comment_count = data.get('comment_count', 0)  # 总评论数
                base_comment_count_l1 = data.get('comment_count_l1', 0)  # 一级评论数
                comments = data.get('comments', [])  # 评论数组
                
                # 4. 遍历每条评论，提取目标字段
                for comment in comments:
                    # 4.1 提取评论基础字段
                    comment_id = comment.get('id', '')
                    comment_type = comment.get('comment_type', '')
                    text_language_code = comment.get('text_language_code', '')
                    time_stamp = comment.get('time', 0)  # 时间戳（秒级）
                    note_id = comment.get('note_id', '')
                    like_count = comment.get('like_count', 0)
                    score = comment.get('score', 0)
                    content = comment.get('content', '')  # 评论内容
                    sub_comment_count = comment.get('sub_comment_count', 0)
                    
                    # 4.2 转换时间戳为日期字符串（格式：YYYY-MM-DD HH:MM:SS）
                    if isinstance(time_stamp, (int, float)) and time_stamp > 0:
                        comment_time = datetime.fromtimestamp(time_stamp).strftime("%Y-%m-%d %H:%M:%S")
                    else:
                        comment_time = "未知时间"
                    
                    # 4.3 提取评论者信息（user子对象）
                    user = comment.get('user', {})
                    user_userid = user.get('userid', '')  # 用户唯一ID
                    user_nickname = user.get('nickname', '')  # 用户昵称
                    user_images = user.get('images', '')  # 用户头像URL
                    
                    # 4.4 整合当前评论的所有字段（与data层字段关联）
                    comment_row = {
                        # data层字段（每个评论关联同一个data的基础信息）
                        "data_user_id": base_user_id,
                        "总评论数_comment_count": base_comment_count,
                        "一级评论数_comment_count_l1": base_comment_count_l1,
                        # 评论层字段
                        "评论ID_comment_id": comment_id,
                        "评论类型_comment_type": comment_type,
                        "语言编码_text_language_code": text_language_code,
                        "评论时间_time": comment_time,
                        "关联笔记ID_note_id": note_id,
                        "点赞数_like_count": like_count,
                        "评论分数_score": score,
                        "评论内容_content": content,
                        "子评论数_sub_comment_count": sub_comment_count,
                        # 用户子对象字段
                        "用户ID_user_userid": user_userid,
                        "用户昵称_user_nickname": user_nickname,
                        "用户头像URL_user_images": user_images
                    }
                    all_data.append(comment_row)
            
            # 处理异常情况（避免单个文件错误导致脚本崩溃）
            except Exception as e:
                print(f"错误: 处理 {filename} 时失败，原因: {str(e)}\n")
                continue
    
    # 5. 将数据转为DataFrame并导出Excel
    if all_data:
        df = pd.DataFrame(all_data)
        # 保存Excel（index=False 不保留行索引）
        df.to_excel(output_file, index=False, engine='openpyxl')
        print(f"\n数据提取完成！共提取 {len(all_data)} 条评论，已保存至: {output_file}")
    else:
        print("\n未提取到任何有效数据，请检查文件路径和文件内容。")

# -------------------------- 请根据实际情况修改以下参数 --------------------------
if __name__ == "__main__":
    # 1. 目标文件根路径（替换为你的文件所在文件夹路径，Windows用r""避免转义）
    ROOT_PATH = r"C:\cao\code\1029\朱迪尼克\评论"  
    # 2. 输出Excel路径（替换为你想保存的Excel路径）
    OUTPUT_EXCEL = "朱迪尼克_评论.xlsx"  
    
    # 执行数据提取和导出
    extract_response_data(ROOT_PATH, OUTPUT_EXCEL)

正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61714] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61752] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61764] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61789] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61798] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61817] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61835] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61848] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61855] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61886] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61897] response_edith.xiaohongshu.com_message.txt
正在处理文件: C:\cao\code\1029\朱迪尼克\评论\[61914] response_edith.xiaohongshu.com_mess

In [4]:
import os
import json
import pandas as pd

def timestamp_to_datetime(ts):
    """
    将时间戳转换为标准日期格式（兼容10位秒级/13位毫秒级时间戳）
    :param ts: 时间戳（int/str）
    :return: 格式化日期字符串（YYYY-MM-DD HH:MM:SS）或空字符串
    """
    if not ts:
        return ""
    ts_str = str(ts).strip()
    try:
        ts_int = int(ts_str)
        # 区分秒级（10位）和毫秒级（13位）时间戳
        if len(ts_str) == 10:
            return pd.to_datetime(ts_int, unit="s").strftime("%Y-%m-%d %H:%M:%S")
        elif len(ts_str) == 13:
            return pd.to_datetime(ts_int, unit="ms").strftime("%Y-%m-%d %H:%M:%S")
        else:
            return f"无效时间戳格式: {ts_str}"
    except Exception as e:
        return f"时间戳转换失败: {str(e)}"

def extract_response_notes(target_dir, output_xlsx):
    """
    提取指定目录下含"response"的文件中的笔记数据
    :param target_dir: 目标文件夹路径
    :param output_xlsx: 输出Excel文件名
    """
    # 存储所有笔记数据的列表
    all_notes_data = []
    
    # 1. 遍历目标目录，筛选含"response"的文件（不区分大小写）
    for filename in os.listdir(target_dir):
        if "response" in filename.lower():
            file_path = os.path.join(target_dir, filename)
            print(f"正在处理文件: {filename}")
            
            try:
                # 2. 读取文件内容（处理HTTP响应头+JSON混合格式）
                with open(file_path, "r", encoding="utf-8") as f:
                    file_content = f.read()
                
                # 提取JSON部分（从第一个"{"开始，忽略前面的HTTP响应头）
                json_start_idx = file_content.find("{")
                if json_start_idx == -1:
                    print(f"警告: {filename} 中未找到JSON数据，跳过该文件\n")
                    continue
                json_content = file_content[json_start_idx:]
                
                # 3. 解析JSON数据
                json_data = json.loads(json_content)
                # 检查数据结构是否合法（需包含data->items）
                if not json_data.get("success") or "data" not in json_data or "items" not in json_data["data"]:
                    print(f"警告: {filename} 中无有效items数据，跳过该文件\n")
                    continue
                notes_items = json_data["data"]["items"]
                
                # 4. 遍历每个笔记item，提取目标字段
                for item in notes_items:
                    # 先获取note子对象（所有信息的根）
                    note_obj = item.get("note", {})
                    if not note_obj:  # 跳过无note数据的item
                        continue

                    # 4.1 笔记基础信息（从note_obj提取）
                    note_base = {
                        "笔记ID": note_obj.get("id", ""),
                        "笔记类型(type)": note_obj.get("type", ""),
                        "笔记标题(title)": note_obj.get("title", ""),
                        "笔记描述(desc)": note_obj.get("desc", ""),
                        "笔记创建时间(timestamp)": timestamp_to_datetime(note_obj.get("timestamp")),
                        "笔记更新时间(update_time)": timestamp_to_datetime(note_obj.get("update_time"))
                    }

                    # 4.2 作者信息（从note_obj["user"]提取）
                    author_info = {
                        "作者userid": note_obj.get("user", {}).get("userid", ""),
                        "作者nickname": note_obj.get("user", {}).get("nickname", ""),
                        "作者头像URL(images)": note_obj.get("user", {}).get("images", "")
                    }

                    # 4.3 互动数据（从note_obj直接提取）
                    interaction_info = {
                        "互动_点赞数": note_obj.get("liked_count", 0),
                        "互动_收藏数": note_obj.get("collected_count", 0),
                        "互动_评论数": note_obj.get("comments_count", 0),
                        "互动_分享数": note_obj.get("shared_count", 0)
                    }

                    # 4.4 合并数据（无share_info，可删除或留空）
                    single_note = {**note_base, **author_info, **interaction_info}
                    all_notes_data.append(single_note)
                
                print(f"成功提取 {filename} 中的 {len(notes_items)} 条笔记数据\n")
            
            # 捕获文件处理中的异常（避免脚本崩溃）
            except UnicodeDecodeError:
                print(f"错误: {filename} 编码格式异常（非UTF-8），跳过该文件\n")
            except json.JSONDecodeError:
                print(f"错误: {filename} JSON解析失败，跳过该文件\n")
            except Exception as e:
                print(f"错误: 处理 {filename} 时发生未知错误 - {str(e)}，跳过该文件\n")
    
    # 5. 转换为DataFrame并保存为Excel
    if all_notes_data:
        df = pd.DataFrame(all_notes_data)
        # 使用openpyxl引擎保存（支持xlsx格式）
        df.to_excel(output_xlsx, index=False, engine="openpyxl")
        print(f"数据提取完成！共获取 {len(df)} 条笔记数据，已保存到 {output_xlsx}")
    else:
        print("未提取到任何有效笔记数据，请检查目标文件夹和文件格式")

# -------------------------- 配置参数（用户可修改）--------------------------
TARGET_DIRECTORY = r"C:\cao\code\1029\服美役\帖子\未命名文件夹2"  # 目标文件夹路径（含response文件的目录）
OUTPUT_EXCEL = "服美役.xlsx"  # 输出Excel文件名
# --------------------------------------------------------------------------

if __name__ == "__main__":
    # 检查目标文件夹是否存在
    if not os.path.exists(TARGET_DIRECTORY):
        print(f"错误: 目标文件夹 {TARGET_DIRECTORY} 不存在，请检查路径")
    else:
        # 安装依赖提示（首次运行若报错需执行）
        try:
            import openpyxl
        except ImportError:
            print("提示: 缺少openpyxl依赖，需先执行命令安装：pip install openpyxl pandas")
        else:
            extract_response_notes(TARGET_DIRECTORY, OUTPUT_EXCEL)

正在处理文件: [56394] response_so.xiaohongshu.com_message.txt
成功提取 [56394] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56405] response_so.xiaohongshu.com_message.txt
成功提取 [56405] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56414] response_so.xiaohongshu.com_message.txt
成功提取 [56414] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56433] response_so.xiaohongshu.com_message.txt
成功提取 [56433] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56462] response_so.xiaohongshu.com_message.txt
成功提取 [56462] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56491] response_so.xiaohongshu.com_message.txt
成功提取 [56491] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56522] response_so.xiaohongshu.com_message.txt
成功提取 [56522] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56557] response_so.xiaohongshu.com_message.txt
成功提取 [56557] response_so.xiaohongshu.com_message.txt 中的 20 条笔记数据

正在处理文件: [56594] response